<a href="https://colab.research.google.com/github/NhanPhamQuang/AI-Driven-Optimization-of-Bus-Scheduling-in-Ho-Chi-Minh-City/blob/main/T%E1%BB%95ng_h%E1%BB%A3p.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

# Crawl bằng API

In [ ]:
import requests
import json
import csv
from datetime import datetime

base_url = "https://api.xebuyt.net/businfo/getvarsbyroute/{}_1"

# Danh sách lưu kết quả chi tiết (mỗi chiều là 1 dòng)
detailed_results = []

print("BẮT ĐẦU LẤY THÔNG TIN CHI TIẾT CÁC TUYẾN...\n")

for route_id in range(1, 6):
    print("=" * 80)
    print(f"Đang request RouteId {route_id}")

    url = base_url.format(route_id)
    try:
        response = requests.get(url, timeout=8)
        print(f"HTTP {response.status_code}")

        if response.status_code == 200:
            text = response.text.strip()
            if text not in ["", "null", "[]", "{}"]:
                try:
                    data = response.json()
                    print(json.dumps(data, indent=2, ensure_ascii=False))

                    # === DUYỆT TỪNG CHIỀU (RouteVar) ===
                    for var in data:
                        detailed_results.append({
                            "RouteId": var.get("RouteId"),
                            "RouteNo": var.get("RouteNo"),
                            "RouteVarId": var.get("RouteVarId"),
                            "RouteVarName": var.get("RouteVarName"),
                            "RouteVarShortName": var.get("RouteVarShortName"),
                            "StartStop": var.get("StartStop"),
                            "EndStop": var.get("EndStop"),
                            "Distance": var.get("Distance"),
                            "RunningTime": var.get("RunningTime"),
                            "Outbound": var.get("Outbound")
                        })
                except json.JSONDecodeError:
                    print("(Không phải JSON hợp lệ)")
                    print(text)
            else:
                print("(Rỗng hoặc không có dữ liệu)")
        else:
            print(f"(Mã lỗi HTTP: {response.status_code})")

    except requests.exceptions.RequestException as e:
        print(f"Lỗi khi request RouteId {route_id}: {e}")

# === GHI KẾT QUẢ RA FILE CSV ===
csv_filename = "bus_routes_api.csv"

fieldnames = [
    "RouteId", "RouteNo", "RouteVarId", "RouteVarName", "RouteVarShortName",
    "StartStop", "EndStop", "Distance", "RunningTime", "Outbound"
]

with open(csv_filename, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(detailed_results)

# === TỔNG KẾT ===
total_routes = len(set(r["RouteId"] for r in detailed_results))
total_vars = len(detailed_results)

print("\n" + "=" * 80)
print("TỔNG KẾT")
print(f"Tổng số tuyến: {total_routes}")
print(f"Tổng số chiều (RouteVar): {total_vars}")
print(f"Chi tiết đã được lưu vào: {csv_filename}")

BẮT ĐẦU LẤY THÔNG TIN CHI TIẾT CÁC TUYẾN...

Đang request RouteId 1
HTTP 200
[
  {
    "Distance": "7680",
    "EndStop": "Bến xe Chợ Lớn",
    "Outbound": "true",
    "RouteId": "1",
    "RouteNo": "1",
    "RouteVarId": "1",
    "RouteVarName": "Lượt đi: Bến Thành - BX Chợ Lớn",
    "RouteVarShortName": "BX Chợ Lớn",
    "RunningTime": "35",
    "StartStop": "Công Trường Mê Linh"
  },
  {
    "Distance": "9534",
    "EndStop": "Công Trường Mê Linh",
    "Outbound": "false",
    "RouteId": "1",
    "RouteNo": "1",
    "RouteVarId": "2",
    "RouteVarName": "Lượt về: BX Chợ Lớn - Bến Thành",
    "RouteVarShortName": "Bến Thành",
    "RunningTime": "30",
    "StartStop": "Bến xe Chợ Lớn"
  }
]
Đang request RouteId 2
HTTP 200
[
  {
    "Distance": "13501",
    "EndStop": "Bến xe Miền Tây",
    "Outbound": "true",
    "RouteId": "2",
    "RouteNo": "2",
    "RouteVarId": "3",
    "RouteVarName": "Lượt đi: Bến Thành - BX Miền Tây",
    "RouteVarShortName": "BX Miền Tây",
    "RunningTime":

In [ ]:
import requests
import json
import csv
import time
from datetime import datetime

# === CẤU HÌNH ===
CSV_INPUT = "bus_routes_api.csv"
OUTPUT_FILE = f"bus_stop_api_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

# API
GET_STOPS_URL = "https://api.xebuyt.net/businfo/getstopsbyvar/{}_1/{}"

# Danh sách lưu toàn bộ trạm
all_stops = []

print("BẮT ĐẦU LẤY TRẠM DỪNG TỪ CSV...\n")

# === 1. ĐỌC CSV → LẤY CẶP (RouteId, RouteVarId) ===
route_vars = []
try:
    with open(CSV_INPUT, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        print(f"Các cột: {reader.fieldnames}")

        for row in reader:
            route_id = row.get("RouteId")
            route_var_id = row.get("RouteVarId")
            route_var_name = row.get("RouteVarName", "")

            if route_id and route_var_id:
                try:
                    route_vars.append({
                        "RouteId": int(route_id),
                        "RouteVarId": int(route_var_id),
                        "RouteVarName": route_var_name
                    })
                except ValueError:
                    continue
except FileNotFoundError:
    print(f"Không tìm thấy file: {CSV_INPUT}")
    exit()

# Loại trùng
unique_vars = { (rv["RouteId"], rv["RouteVarId"]): rv for rv in route_vars }
route_vars = list(unique_vars.values())

print(f"Tìm thấy {len(route_vars)} chiều tuyến để lấy trạm dừng.\n")

# === 2. DUYỆT TỪNG CHIỀU → LẤY TRẠM DỪNG ===
for idx, rv in enumerate(route_vars, 1):
    route_id = rv["RouteId"]
    route_var_id = rv["RouteVarId"]
    route_var_name = rv["RouteVarName"]

    print(f"[{idx}/{len(route_vars)}] [{route_id} | Var {route_var_id}] {route_var_name}")

    url = GET_STOPS_URL.format(route_id, route_var_id)
    print(f"   → Gọi: {url}")

    try:
        response = requests.get(url, timeout=12)
        if response.status_code == 200:
            try:
                stops_data = response.json()
                if isinstance(stops_data, list):
                    # Gắn thêm thông tin tuyến
                    for stop in stops_data:
                        stop["RouteId"] = route_id
                        stop["RouteVarId"] = route_var_id
                        stop["RouteVarName"] = route_var_name
                    all_stops.extend(stops_data)
                    print(f"   Thành công! {len(stops_data)} trạm")
                else:
                    print("   Dữ liệu không phải danh sách")
            except json.JSONDecodeError as e:
                print(f"   JSON lỗi: {e}")
                print(response.text[:200])
        else:
            print(f"   HTTP {response.status_code}")

    except Exception as e:
        print(f"   Lỗi: {e}")

    time.sleep(0.5)  # An toàn với server

# === 3. GHI RA FILE JSON ===
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(all_stops, f, indent=2, ensure_ascii=False)

print("\n" + "="*80)
print("HOÀN TẤT!")
print(f"Tổng cộng: {len(all_stops)} trạm dừng")
print(f"Đã lưu vào: {OUTPUT_FILE}")

BẮT ĐẦU LẤY TRẠM DỪNG TỪ CSV...

Các cột: ['RouteId', 'RouteNo', 'RouteVarId', 'RouteVarName', 'RouteVarShortName', 'StartStop', 'EndStop', 'Distance', 'RunningTime', 'Outbound']
Tìm thấy 10 chiều tuyến để lấy trạm dừng.

[1/10] [1 | Var 1] Lượt đi: Bến Thành - BX Chợ Lớn
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/1_1/1
   Thành công! 29 trạm
[2/10] [1 | Var 2] Lượt về: BX Chợ Lớn - Bến Thành
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/1_1/2
   Thành công! 35 trạm
[3/10] [2 | Var 3] Lượt đi: Bến Thành - BX Miền Tây
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/2_1/3
   Thành công! 43 trạm
[4/10] [2 | Var 4] Lượt về: BX Miền Tây - Bến Thành
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/2_1/4
   Thành công! 37 trạm
[5/10] [3 | Var 5] Lượt đi: Bến Thành - Thạnh Lộc
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/3_1/5
   Thành công! 48 trạm
[6/10] [3 | Var 6] Lượt về: Thạnh Lộc - Bến Thành
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyv

In [ ]:
import requests
import json
import csv
import time
from datetime import datetime

# === CẤU HÌNH ===
CSV_INPUT = "bus_routes_api.csv"
CSV_OUTPUT = f"bus_stops_detailed_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

# API
GET_STOPS_URL = "https://api.xebuyt.net/businfo/getstopsbyvar/{}_1/{}"

# Danh sách lưu toàn bộ trạm
all_stops = []

print("BẮT ĐẦU LẤY TRẠM DỪNG TỪ CSV VÀ XUẤT RA CSV...\n")

# === 1. ĐỌC CSV → LẤY CẶP (RouteId, RouteVarId) ===
route_vars = []
try:
    with open(CSV_INPUT, mode='r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        print(f"Các cột trong file đầu vào: {reader.fieldnames}")

        for row in reader:
            route_id = row.get("RouteId")
            route_var_id = row.get("RouteVarId")
            route_var_name = row.get("RouteVarName", "")
            route_no = row.get("RouteNo", "")

            if route_id and route_var_id:
                try:
                    route_vars.append({
                        "RouteId": int(route_id),
                        "RouteVarId": int(route_var_id),
                        "RouteVarName": route_var_name,
                        "RouteNo": route_no
                    })
                except ValueError:
                    continue
except FileNotFoundError:
    print(f"Không tìm thấy file: {CSV_INPUT}")
    exit()

# Loại trùng (RouteId + RouteVarId)
unique_vars = { (rv["RouteId"], rv["RouteVarId"]): rv for rv in route_vars }
route_vars = list(unique_vars.values())

print(f"Tìm thấy {len(route_vars)} chiều tuyến để lấy trạm dừng.\n")

# === 2. DUYỆT TỪNG CHIỀU → LẤY TRẠM DỪNG ===
for idx, rv in enumerate(route_vars, 1):
    route_id = rv["RouteId"]
    route_var_id = rv["RouteVarId"]
    route_var_name = rv["RouteVarName"]
    route_no = rv["RouteNo"]

    print(f"[{idx}/{len(route_vars)}] Tuyến {route_no} | {route_var_name}")

    url = GET_STOPS_URL.format(route_id, route_var_id)
    print(f"   → Gọi: {url}")

    try:
        response = requests.get(url, timeout=12)
        if response.status_code == 200:
            try:
                stops_data = response.json()
                if isinstance(stops_data, list):
                    # Gắn thêm thông tin tuyến
                    for stop in stops_data:
                        stop["RouteId"] = route_id
                        stop["RouteVarId"] = route_var_id
                        stop["RouteVarName"] = route_var_name
                        stop["RouteNo"] = route_no
                    all_stops.extend(stops_data)
                    print(f"   Thành công! {len(stops_data)} trạm")
                else:
                    print("   Dữ liệu không phải danh sách")
            except json.JSONDecodeError as e:
                print(f"   JSON lỗi: {e}")
        else:
            print(f"   HTTP {response.status_code}")

    except Exception as e:
        print(f"   Lỗi: {e}")

    time.sleep(0.5)

# === 3. GHI RA FILE CSV ===
fieldnames = [
    "RouteNo", "RouteId", "RouteVarId", "RouteVarName",
    "StopID", "Name", "AddressNo", "Lat", "Lng",
    "Code", "Street", "StopType", "Status", "Routes", "Search"
]

with open(CSV_OUTPUT, 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for stop in all_stops:
        row = {key: stop.get(key, "") for key in fieldnames}
        writer.writerow(row)

print("\n" + "="*80)
print("HOÀN TẤT!")
print(f"Tổng cộng: {len(all_stops)} trạm dừng")
print(f"Đã lưu file CSV: {CSV_OUTPUT}")
print("   → Mở bằng Excel, Google Sheets được ngay!")

BẮT ĐẦU LẤY TRẠM DỪNG TỪ CSV VÀ XUẤT RA CSV...

Các cột trong file đầu vào: ['RouteId', 'RouteNo', 'RouteVarId', 'RouteVarName', 'RouteVarShortName', 'StartStop', 'EndStop', 'Distance', 'RunningTime', 'Outbound']
Tìm thấy 10 chiều tuyến để lấy trạm dừng.

[1/10] Tuyến 1 | Lượt đi: Bến Thành - BX Chợ Lớn
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/1_1/1
   Thành công! 29 trạm
[2/10] Tuyến 1 | Lượt về: BX Chợ Lớn - Bến Thành
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/1_1/2
   Thành công! 35 trạm
[3/10] Tuyến 2 | Lượt đi: Bến Thành - BX Miền Tây
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/2_1/3
   Thành công! 43 trạm
[4/10] Tuyến 2 | Lượt về: BX Miền Tây - Bến Thành
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/2_1/4
   Thành công! 37 trạm
[5/10] Tuyến 3 | Lượt đi: Bến Thành - Thạnh Lộc
   → Gọi: https://api.xebuyt.net/businfo/getstopsbyvar/3_1/5
   Thành công! 48 trạm
[6/10] Tuyến 3 | Lượt về: Thạnh Lộc - Bến Thành
   → Gọi: https://api.xebuyt.n

# Crawl bằng HTML

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd

# URL của trang web
url = 'https://xebuyt.net/tuyen-xe-buyt'

# Gửi request để lấy nội dung trang
response = requests.get(url)
response.raise_for_status()  # Kiểm tra lỗi nếu có

# Parse HTML với BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Tìm div chứa danh sách tuyến
div_result = soup.find('div', id='divResult')

# Tìm tất cả các thẻ <a> với class 'cms-button'
route_links = div_result.find_all('a', class_='cms-button')

# Danh sách để lưu dữ liệu (chỉ lấy tối đa 5 tuyến đầu tiên)
data = []

# Duyệt qua tối đa 5 link đầu tiên
for link in route_links[:5]:  # <-- Chỉ lấy 5 phần tử đầu
    # Lấy href
    href = link.get('href')
    full_url = 'https://xebuyt.net' + href if href.startswith('/') else href

    # Lấy số tuyến từ span.icon
    route_number = link.find('span', class_='icon').get_text(strip=True)

    # Lấy tên tuyến từ div.routetrip
    route_name = link.find('div', class_='routetrip').get_text(strip=True)

    # Thêm vào danh sách
    data.append({
        'Route_Number': route_number,
        'Route_Name': route_name,
        'URL': full_url
    })

# Lưu vào file CSV
filename = 'tuyen_xe_buyt.csv'
with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Route_Number', 'Route_Name', 'URL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    writer.writerows(data)

print(f'Đã lưu 5 tuyến xe buýt đầu tiên vào file {filename}. Tổng số tuyến: {len(data)}')

# Hiển thị dưới dạng DataFrame (tùy chọn)
df = pd.DataFrame(data)
df

Đã lưu 5 tuyến xe buýt đầu tiên vào file tuyen_xe_buyt.csv. Tổng số tuyến: 5


,Route_Number,Route_Name,URL
0,01,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html
1,02,Bến Thành- BX Miền Tây,https://xebuyt.net/ben-thanh-ben-xe-mien-tay.html
2,03,Bến Thành- Thạnh Lộc,https://xebuyt.net/ben-thanh-thanh-loc.html
3,04,Bến Thành- Cộng Hòa- An Sương,https://xebuyt.net/ben-thanh-cong-hoa-an-suong...
4,05,Bến xe Chợ Lớn - Biên Hòa,https://xebuyt.net/ben-xe-cho-lon-bien-hoa.html


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
df = pd.read_csv(csv_filename)

# Danh sách để lưu dữ liệu mở rộng
extended_data = []

# Headers cho CSV mới
headers = [
    'Route_Number', 'Route_Name', 'URL',
    'Operator', 'Operating_Type', 'Distance', 'Vehicle_Type', 'Operating_Hours',
    'Ticket_Price', 'Number_of_Trips', 'Trip_Time', 'Frequency'
]

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"Đang scrape tuyến {route_number}: {route_name}")

    try:
        # Gửi request
        response = requests.get(url)
        response.raise_for_status()

        # Parse HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Tìm div rouInfo và table
        rou_info_div = soup.find('div', id='rouInfo')
        if not rou_info_div:
            print(f"Không tìm thấy div#rouInfo cho tuyến {route_number}")
            continue

        table = rou_info_div.find('table', class_='tbl100')
        if not table:
            print(f"Không tìm thấy table cho tuyến {route_number}")
            continue

        # Khởi tạo giá trị mặc định
        operator = ''
        operating_type = ''
        distance = ''
        vehicle_type = ''
        operating_hours = ''
        ticket_price = ''
        number_of_trips = ''
        trip_time = ''
        frequency = ''

        # Duyệt qua các hàng trong table
        rows = table.find('tbody').find_all('tr')
        for tr in rows:
            tds = tr.find_all('td')
            if len(tds) < 2:
                continue

            label_td = tds[0].get_text(strip=True)
            content = tds[1].get_text(separator='\n').strip()

            # Xử lý Đơn vị đảm nhận (có thể ở hàng riêng)
            if 'Đơn vị đảm nhận' in label_td:
                operator = content

            # Bỏ qua hàng spacing
            if not label_td and not content:
                continue

            # Xử lý ul lists ở hàng cuối (cột 2 và 3)
            if len(tds) >= 3 and not label_td:  # Hàng có ul lists
                left_ul = tds[1].find('ul')
                right_ul = tds[2].find('ul') if len(tds) > 2 else None

                if left_ul:
                    left_items = [li.get_text(strip=True) for li in left_ul.find_all('li')]
                    for item in left_items:
                        if 'Loại hình hoạt động' in item:
                            operating_type = item.replace('Loại hình hoạt động:', '').strip()
                        elif 'Cự ly' in item:
                            distance = item.replace('Cự ly:', '').strip()
                        elif 'Loại xe' in item:
                            vehicle_type = item.replace('Loại xe:', '').strip()
                        elif 'Thời gian hoạt động' in item:
                            operating_hours = item.replace('Thời gian hoạt động:', '').strip()

                if right_ul:
                    right_items = [li.get_text(separator='\n').strip() for li in right_ul.find_all('li')]
                    for item in right_items:
                        if 'Giá vé' in item:
                            ticket_price = item.replace('Giá vé:', '').strip()
                        elif 'Số chuyến' in item:
                            number_of_trips = item.replace('Số chuyến:', '').strip()
                        elif 'Thời gian chuyến' in item:
                            trip_time = item.replace('Thời gian chuyến:', '').strip()
                        elif 'Giãn cách chuyến' in item:
                            frequency = item.replace('Giãn cách chuyến:', '').strip()

        # Thêm vào danh sách
        extended_data.append({
            'Route_Number': route_number,
            'Route_Name': route_name,
            'URL': url,
            'Operator': operator,
            'Operating_Type': operating_type,
            'Distance': distance,
            'Vehicle_Type': vehicle_type,
            'Operating_Hours': operating_hours,
            'Ticket_Price': ticket_price,
            'Number_of_Trips': number_of_trips,
            'Trip_Time': trip_time,
            'Frequency': frequency
        })

    except Exception as e:
        print(f"Lỗi khi scrape tuyến {route_number}: {e}")
        # Vẫn thêm dữ liệu với giá trị rỗng
        extended_data.append({
            'Route_Number': route_number,
            'Route_Name': route_name,
            'URL': url,
            'Operator': '',
            'Operating_Type': '',
            'Distance': '',
            'Vehicle_Type': '',
            'Operating_Hours': '',
            'Ticket_Price': '',
            'Number_of_Trips': '',
            'Trip_Time': '',
            'Frequency': ''
        })

    # Delay để tránh overload server
    time.sleep(1)

# Tạo DataFrame và lưu vào CSV mới
extended_df = pd.DataFrame(extended_data)
output_filename = 'tuyen_xe_buyt_detailed.csv'
extended_df.to_csv(output_filename, index=False, encoding='utf-8')

print(f'Dữ liệu chi tiết đã được lưu vào file {output_filename}. Tổng số tuyến: {len(extended_data)}')

# Hiển thị DataFrame (comment nếu không cần)
extended_df.head()

Đang scrape tuyến 1: Bến Thành- BX Chợ Lớn
Đang scrape tuyến 2: Bến Thành- BX Miền Tây
Đang scrape tuyến 3: Bến Thành- Thạnh Lộc
Đang scrape tuyến 4: Bến Thành- Cộng Hòa- An Sương
Đang scrape tuyến 5: Bến xe Chợ Lớn - Biên Hòa
Dữ liệu chi tiết đã được lưu vào file tuyen_xe_buyt_detailed.csv. Tổng số tuyến: 5


,Route_Number,Route_Name,URL,Operator,Operating_Type,Distance,Vehicle_Type,Operating_Hours,Ticket_Price,Number_of_Trips,Trip_Time,Frequency
0,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,"Công ty Cổ phần Xe khách Sài Gòn, ĐT: (08) 38....",Phổ thông - Có trợ giá,8.6 km,80 chỗ,05:00 - 20:30,"&nbsp- Vé lượt: 5,000 VNĐ\n &nbsp- Vé lượt...",240 chuyến/ngày,30 - 35 phút,6 - 10 phút
1,2,Bến Thành- BX Miền Tây,https://xebuyt.net/ben-thanh-ben-xe-mien-tay.html,"Công ty TNHH Vận tải Ngôi sao Sài Gòn, ĐT: (08...",Phổ thông - Có trợ giá,13.65 km,30 - 80 chỗ,04:45 - 18:30,"&nbsp- Vé lượt: 5,000 VNĐ\n &nbsp- Vé lượt...",180 chuyến/ngày,45 - 50 phút,8 - 12 phút
2,3,Bến Thành- Thạnh Lộc,https://xebuyt.net/ben-thanh-thanh-loc.html,"Công ty TNHH Vận tải Ngôi sao Sài Gòn, ĐT: (08...",Phổ thông - Có trợ giá,18.05 km,26 - 55 chỗ,04:15 - 20:45,"&nbsp- Vé lượt: 6,000 VNĐ\n &nbsp- Vé lượt...",312 chuyến/ngày,55 - 65 phút,4 - 12 phút
3,4,Bến Thành- Cộng Hòa- An Sương,https://xebuyt.net/ben-thanh-cong-hoa-an-suong...,"Công ty TNHH Vận tải Ngôi sao Sài Gòn, ĐT: (08...",Phổ thông - Có trợ giá,16.6 km,30 - 55 chỗ,05:00 - 20:30,"&nbsp- Vé lượt: 5,000 VNĐ\n &nbsp- Vé lượt...",303 chuyến/ngày,50 - 65 phút,5 - 10 phút
4,5,Bến xe Chợ Lớn - Biên Hòa,https://xebuyt.net/ben-xe-cho-lon-bien-hoa.html,"Hợp tác xã vận tải xe buýt Quyết Thắng, ĐT: (0...",Phổ thông - Không trợ giá,38 km,30 - 80 chỗ,04:50 - 17:50,,72 chuyến/ngày,60 - 95 phút,20 - 25 phút


In [ ]:
extended_df.columns

Index(['Route_Number', 'Route_Name', 'URL', 'Operator', 'Operating_Type',
       'Distance', 'Vehicle_Type', 'Operating_Hours', 'Ticket_Price',
       'Number_of_Trips', 'Trip_Time', 'Frequency'],
      dtype='object')

In [ ]:
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd
import time
from pprint import pprint  # Để debug nếu cần

# Đọc file CSV đã tạo trước đó
csv_filename = 'tuyen_xe_buyt.csv'
try:
    df = pd.read_csv(csv_filename)
    print(f"Đã đọc thành công {len(df)} tuyến từ file {csv_filename}")
except FileNotFoundError:
    print(f"Không tìm thấy file {csv_filename}. Vui lòng chạy code scrape URL trước!")
    exit()

# Danh sách để lưu dữ liệu lộ trình (mỗi stop là một row)
route_stops_data = []

# Giới hạn số tuyến để test (bỏ comment nếu muốn scrape tất cả)
# df = df.head(5)  # Chỉ scrape 5 tuyến đầu để test

print(f"Bắt đầu scrape lộ trình cho {len(df)} tuyến...")

# Duyệt qua từng hàng trong DataFrame
for index, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"\n--- Đang scrape lộ trình tuyến {route_number}: {route_name} ---")
    print(f"URL: {url}")

    try:
        # Gửi yêu cầu HTTP để lấy nội dung trang web
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Kiểm tra lỗi HTTP

        # Giải mã HTML để xử lý các ký tự đặc biệt
        html_content = html.unescape(response.text)

        # Sử dụng BeautifulSoup để phân tích HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        print(f"✅ Đã parse HTML thành công. Độ dài content: {len(response.text)}")

        # Khởi tạo danh sách stops cho tuyến này
        stops_data = []

        # Biến để xác định hướng hiện tại
        current_direction = None

        # Duyệt qua các element h3 và table theo thứ tự
        for element in soup.find_all(['h3', 'table']):
            if element.name == 'h3':
                # Cập nhật hướng dựa trên tiêu đề (tìm các pattern phổ biến như "Đi [Điểm cuối]")
                header_text = element.get_text(strip=True)
                if "Đi" in header_text:
                    # Lấy phần sau "Đi " làm tên hướng
                    direction_name = header_text.replace("Đi ", "").strip()
                    current_direction = direction_name
                    print(f"  ✅ Phát hiện hướng mới: {current_direction}")
                else:
                    current_direction = None
            elif element.name == 'table' and current_direction:
                # Tìm tất cả các hàng trong bảng
                rows = element.find_all('tr')
                stop_count = 0
                for row in rows:
                    cols = row.find_all('td')
                    if len(cols) >= 2:
                        # Lấy thứ tự stop (từ cột 1, thường là <div> với A/1/2/.../B)
                        order_col = cols[0].find('div')
                        stop_order = order_col.get_text(strip=True) if order_col else str(stop_count + 1)

                        # Lấy tên stop từ cột 2
                        stop_name = cols[1].get_text(strip=True)

                        if stop_name and stop_name != '':  # Bỏ qua stop rỗng
                            # Thêm vào danh sách
                            stops_data.append({
                                'Route_Number': route_number,
                                'Route_Name': route_name,
                                'URL': url,
                                'Direction': current_direction,
                                'Stop_Order': stop_order,
                                'Stop_Name': stop_name
                            })
                            stop_count += 1
                            if stop_count <= 3:  # Chỉ print 3 stops đầu mỗi hướng để tránh spam
                                print(f"    Stop {stop_order}: {stop_name}")
                            if stop_count == 4:
                                print(f"    ... (và {len(rows) - stop_count + 1} stops khác)")

                print(f"  ✅ Thu thập {stop_count} stops cho hướng '{current_direction}'")

        # Thêm stops của tuyến này vào danh sách tổng
        route_stops_data.extend(stops_data)
        total_stops_this_route = len(stops_data)
        print(f"✅ Hoàn thành tuyến {route_number}: {total_stops_this_route} stops tổng cộng")

        # Debug: In preview stops nếu cần (comment nếu không muốn)
        # pprint(stops_data, indent=2)

    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi HTTP cho tuyến {route_number}: {e}")
    except Exception as e:
        print(f"❌ Lỗi khi xử lý dữ liệu tuyến {route_number}: {e}")
        import traceback
        traceback.print_exc()  # In chi tiết lỗi để debug

    # Delay để tránh overload server
    # time.sleep(2)

print(f"\n--- Kết thúc scrape ---")
print(f"Tổng số stops thu thập được: {len(route_stops_data)}")

if len(route_stops_data) == 0:
    print("❌ Không có dữ liệu nào! Kiểm tra log trên để debug.")
    print("Có thể một số tuyến không có lộ trình, hoặc HTML thay đổi.")

# Tạo DataFrame và lưu vào CSV mới
if len(route_stops_data) > 0:
    route_stops_df = pd.DataFrame(route_stops_data)
    output_filename = 'tuyen_xe_buyt_lotrinh.csv'
    route_stops_df.to_csv(output_filename, index=False, encoding='utf-8')

    print(f"✅ Dữ liệu lộ trình đã được lưu vào file {output_filename}")

    # Hiển thị preview DataFrame (top 20 rows)
    print("\n--- Preview DataFrame (top 20 rows) ---")
    print(route_stops_df.head(20).to_string(index=False))

    # Thống kê nhanh
    print(f"\n--- Thống kê ---")
    print(f"- Tổng tuyến có dữ liệu: {route_stops_df['Route_Number'].nunique()}")
    print(f"- Hướng phổ biến nhất: {route_stops_df['Direction'].value_counts().head(1)}")
else:
    print("❌ Không lưu file vì không có dữ liệu.")

Đã đọc thành công 5 tuyến từ file tuyen_xe_buyt.csv
Bắt đầu scrape lộ trình cho 5 tuyến...

--- Đang scrape lộ trình tuyến 1: Bến Thành- BX Chợ Lớn ---
URL: https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html
✅ Đã parse HTML thành công. Độ dài content: 52658
  ✅ Phát hiện hướng mới: BX Chợ Lớn
    Stop A: Công Trường Mê Linh
    Stop 1: Bến Bạch Đằng
    Stop 2: Cục Hải Quan Thành Phố
    ... (và 26 stops khác)
  ✅ Thu thập 29 stops cho hướng 'BX Chợ Lớn'
  ✅ Phát hiện hướng mới: Bến Thành
    Stop A: Bến xe Chợ Lớn
    Stop 1: Chu Văn An
    Stop 2: Tháp Mười
    ... (và 32 stops khác)
  ✅ Thu thập 35 stops cho hướng 'Bến Thành'
  ✅ Phát hiện hướng mới: BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)
  ✅ Thu thập 0 stops cho hướng 'BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)'
    Stop 1: 05:00 - 05:35
    Stop 2: 05:10 - 05:45
    Stop 3: 05:20 - 05:55
    ... (và 117 stops khác)
  ✅ Thu thập 120 stops cho hướng 'BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Th

In [ ]:
import pandas as pd

# Thiết lập để hiển thị toàn bộ DataFrame (tắt giới hạn hàng và cột)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Hiển thị DataFrame

route_stops_df.head(300)

,Route_Number,Route_Name,URL,Direction,Stop_Order,Stop_Name
0,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,A,Công Trường Mê Linh
1,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,1,Bến Bạch Đằng
2,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,2,Cục Hải Quan Thành Phố
3,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,3,Chợ Cũ
4,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,4,Trường Cao Thắng
5,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,5,Công ty Đường sắt
6,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,6,Bến Thành B
7,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,7,Trường Ernst Thalmann
8,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,8,KTX Trần Hưng Đạo
9,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,BX Chợ Lớn,9,Rạp Hưng Đạo


In [ ]:
import re
import pandas as pd

# Tạo timetable_df (nếu chưa có)
if 'timetable_df' not in globals():
    timetable_df = pd.DataFrame(columns=route_stops_df.columns)
    print("✅ Tạo timetable_df")
# 1) Chuẩn hóa cột Stop_Name sang string và thay các loại gạch nối về '-'
route_stops_df['Stop_Name'] = (
    route_stops_df['Stop_Name']
    .astype(str)
    .str.replace(r'[–—−]', '-', regex=True)  # en/em dash, minus
    .str.strip()
)

# 2) Regex nhận diện "biểu đồ giờ": HH:MM - HH:MM (24h), cho phép khoảng trắng linh hoạt
time_range_regex = r'^(?:[01]?\d|2[0-3]):[0-5]\d\s*-\s*(?:[01]?\d|2[0-3]):[0-5]\d$'

mask_is_timetable = route_stops_df['Stop_Name'].str.fullmatch(time_range_regex, na=False)

# 3) Tạo/append sang timetable_df
try:
    timetable_df  # check đã tồn tại chưa
except NameError:
    timetable_df = route_stops_df.loc[mask_is_timetable].copy()
else:
    timetable_df = pd.concat(
        [timetable_df, route_stops_df.loc[mask_is_timetable]],
        ignore_index=True
    )

# 4) Xóa các dòng đã move khỏi route_stops_df
route_stops_df = route_stops_df.loc[~mask_is_timetable].copy()

# (tuỳ chọn) reset index cho đẹp
timetable_df.reset_index(drop=True, inplace=True)
route_stops_df.reset_index(drop=True, inplace=True)



✅ Tạo timetable_df


In [ ]:
timetable_df.head()

,Route_Number,Route_Name,URL,Direction,Stop_Order,Stop_Name
0,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,"BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)",1,05:00 - 05:35
1,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,"BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)",2,05:10 - 05:45
2,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,"BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)",3,05:20 - 05:55
3,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,"BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)",4,05:30 - 06:05
4,1,Bến Thành- BX Chợ Lớn,https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html,"BX Chợ Lớn(Thứ 2, Thứ 3, Thứ 4, Thứ 5, Thứ 6, Thứ 7, Chủ Nhật)",5,05:38 - 06:13


In [ ]:
import requests
from bs4 import BeautifulSoup
import html
import pandas as pd
import re
import time

# ===== Helper: tách info & biểu đồ giờ trong 1 panel (một chiều "Đi ...") =====
def parse_timetable_panel(header_h3):
    """
    Input: thẻ <h3> của một chiều (ví dụ: 'Đi An Sương')
    Output:
        direction (str),
        info (dict): Trips_Count, Trip_Duration, Operating_Time (có thể None nếu không tìm được),
        trips (list[dict]): [{Order, Start, End, Raw}]
    """
    # Lấy tên chiều (text trực tiếp của h3, bỏ phần <span> ngày)
    direction = header_h3.find(string=True, recursive=False)
    direction = direction.strip() if direction else header_h3.get_text(strip=True)

    panel = header_h3.find_next_sibling("div", class_="ui-accordion-content")
    info = {"Trips_Count": None, "Trip_Duration": None, "Operating_Time": None}
    trips = []

    if not panel:
        return direction, info, trips

    # 1) Lấy bảng info đầu panel (nếu có)
    info_table = panel.find("table")
    if info_table:
        for td in info_table.find_all("td"):
            text = td.get_text(strip=True)
            if text.startswith("Số chuyến:"):
                m = re.search(r"Số chuyến:\s*(\d+)", text)
                if m:
                    info["Trips_Count"] = int(m.group(1))
            elif text.startswith("Thời gian chuyến:"):
                # Ví dụ: "Thời gian chuyến: 50 - 65 phút"
                info["Trip_Duration"] = text.split(":", 1)[-1].strip()
            elif text.startswith("Thời gian hoạt động:"):
                # Ví dụ: "Thời gian hoạt động: 05:30 - 20:30"
                info["Operating_Time"] = text.split(":", 1)[-1].strip()

    # 2) Lấy các ô giờ trong panel (td.time: "HH:MM - HH:MM")
    order = 0
    for td in panel.select("td.time"):
        raw = td.get_text(strip=True)
        m = re.search(r"(\d{2}:\d{2})\s*-\s*(\d{2}:\d{2})", raw)
        if m:
            order += 1
            trips.append({
                "Order": order,
                "Start": m.group(1),
                "End": m.group(2),
                "Raw": raw
            })
    return direction, info, trips


# ====== Main: chỉ lấy biểu đồ giờ ======
csv_filename = 'tuyen_xe_buyt.csv'
try:
    df = pd.read_csv(csv_filename)
    print(f"Đã đọc thành công {len(df)} tuyến từ file {csv_filename}")
except FileNotFoundError:
    print(f"Không tìm thấy file {csv_filename}. Vui lòng chạy code scrape URL trước!")
    raise SystemExit

# Giới hạn để test nhanh
df = df.head(5)
print(f"Test với {len(df)} tuyến đầu tiên...")

timetable_rows = []

print(f"Bắt đầu scrape biểu đồ giờ cho {len(df)} tuyến...")
for _, row in df.iterrows():
    route_number = row['Route_Number']
    route_name = row['Route_Name']
    url = row['URL']

    print(f"\n--- Tuyến {route_number}: {route_name} ---")
    print(f"URL: {url}")

    try:
        headers = {
            'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                           'AppleWebKit/537.36 (KHTML, like Gecko) '
                           'Chrome/120.0.0.0 Safari/537.36')
        }
        resp = requests.get(url, headers=headers, timeout=30)
        resp.raise_for_status()

        html_content = html.unescape(resp.text)
        soup = BeautifulSoup(html_content, "html.parser")
        print(f"✅ Parse HTML ok. Độ dài: {len(resp.text)}")

        # Duyệt tất cả chiều trong #accTimetable
        headers_h3 = soup.select("#accTimetable > h3.ui-accordion-header")
        if not headers_h3:
            print("⚠️ Không tìm thấy H3 chiều trong #accTimetable")
            continue

        for h3 in headers_h3:
            # Chỉ giữ các h3 có chữ "Đi "
            if "Đi " not in h3.get_text():
                continue

            direction, info, trips = parse_timetable_panel(h3)

            # Append từng trip
            for t in trips:
                timetable_rows.append({
                    "Route_Number": route_number,
                    "Route_Name": route_name,
                    "URL": url,
                    "Direction": direction,
                    "Order": t["Order"],
                    "Start": t["Start"],
                    "End": t["End"],
                    "Raw": t["Raw"],
                    "Trips_Count": info["Trips_Count"],
                    "Trip_Duration": info["Trip_Duration"],
                    "Operating_Time": info["Operating_Time"],
                })

            print(f"  ✅ {direction}: {len(trips)} chuyến")

    except requests.exceptions.RequestException as e:
        print(f"❌ HTTP error tuyến {route_number}: {e}")
    except Exception as e:
        print(f"❌ Lỗi xử lý tuyến {route_number}: {e}")
        import traceback; traceback.print_exc()

    time.sleep(1.5)

print("\n--- Kết thúc scrape ---")
print(f"Tổng số dòng biểu đồ giờ thu được: {len(timetable_rows)}")

if timetable_rows:
    timetable_df = pd.DataFrame(timetable_rows)
    out_file = "tuyen_xe_buyt_bieudo_gio.csv"
    timetable_df.to_csv(out_file, index=False, encoding="utf-8")
    print(f"✅ Đã lưu {len(timetable_df)} dòng vào {out_file}")

    # Preview
    print("\n--- Preview (top 20) ---")
    print(timetable_df.head(20).to_string(index=False))

    # Thống kê nhanh
    print("\n--- Thống kê ---")
    print(f"- Số tuyến có dữ liệu: {timetable_df['Route_Number'].nunique()}")
    print(f"- Số chiều: {timetable_df['Direction'].nunique()}")
    print(f"- Số chuyến TB/tuyến: {len(timetable_df) / timetable_df['Route_Number'].nunique():.1f}")
else:
    print("❌ Không có dữ liệu biểu đồ giờ. Kiểm tra HTML thực tế của site.")


Đã đọc thành công 5 tuyến từ file tuyen_xe_buyt.csv
Test với 5 tuyến đầu tiên...
Bắt đầu scrape biểu đồ giờ cho 5 tuyến...

--- Tuyến 1: Bến Thành- BX Chợ Lớn ---
URL: https://xebuyt.net/ben-thanh-ben-xe-cho-lon.html
✅ Parse HTML ok. Độ dài: 52658
⚠️ Không tìm thấy H3 chiều trong #accTimetable

--- Tuyến 2: Bến Thành- BX Miền Tây ---
URL: https://xebuyt.net/ben-thanh-ben-xe-mien-tay.html
✅ Parse HTML ok. Độ dài: 52394
⚠️ Không tìm thấy H3 chiều trong #accTimetable

--- Tuyến 3: Bến Thành- Thạnh Lộc ---
URL: https://xebuyt.net/ben-thanh-thanh-loc.html
✅ Parse HTML ok. Độ dài: 67605
⚠️ Không tìm thấy H3 chiều trong #accTimetable

--- Tuyến 4: Bến Thành- Cộng Hòa- An Sương ---
URL: https://xebuyt.net/ben-thanh-cong-hoa-an-suong.html
✅ Parse HTML ok. Độ dài: 65800
⚠️ Không tìm thấy H3 chiều trong #accTimetable

--- Tuyến 5: Bến xe Chợ Lớn - Biên Hòa ---
URL: https://xebuyt.net/ben-xe-cho-lon-bien-hoa.html
✅ Parse HTML ok. Độ dài: 56251
⚠️ Không tìm thấy H3 chiều trong #accTimetable

--- Kế